In [ ]:
pip install hexalattice


In [ ]:
import pygame

In [ ]:
import random

In [ ]:
import math

In [ ]:
from hexalattice.hexalattice import create_hex_grid


## Window and Grid dimensions

In [ ]:
# Window dimensions
WIDTH, HEIGHT = 1000, 750

# Hexagonal grid dimensions
GRID_SIZE = 15

## Set the images for the pokemons and their power structures 

This is to give a behavioural pattern to the pokemon cells being created for their growth and transformation of cells.

In [ ]:
# Pokémon type images
POKEMON_IMAGES = {
    'Fire': 'Charmander.png',
    'Water': 'Squirtle.png',
    'Grass': 'Bulbasaur.png',
    'Flying': 'Pidgeotto.png',
    'Electric': 'Pikachu.png',
    'Rock': 'Geodude.png'
}


# Overpowering structure
OVERPOWER = {
    'Flying': ['Grass', 'Rock'],
    'Water': ['Fire', 'Rock'],
    
    'Fire': ['Grass', 'Flying'],
    'Electric': ['Water', 'Flying'],
    
    'Grass': ['Water', 'Electric'],
    'Rock': ['Fire', 'Electric']
}


In [ ]:
# Initialize Pygame
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()

class Hexagon:
    
    def __init__(self, x, y, image=None):
        self.x = x
        self.y = y
        self.image = image
        self.type = None
        self.colored_by = None
        self.neighbors = []
        
    def draw(self):
        # Draw hexagon border
        points = self.calculate_hexagon_points(self.x, self.y, 20)
        pygame.draw.polygon(screen, (0, 0, 0), points, 2)  # Black border
        
        # Draw hexagon color
        if self.colored_by:
            color = {
                'Water': (0, 0, 255),
                'Flying': (55, 55, 55),
                
                'Fire': (255, 0, 0),
                'Electric': (255, 255, 0),
                
                'Grass': (0, 255, 0),
                'Rock': (252, 205, 200)
            }[self.colored_by]
            pygame.draw.polygon(screen, color, points)

        # Draw hexagon image
        if self.image:
            try:
                image = pygame.image.load(self.image)
                image = pygame.transform.scale(image, (40, 40))
                screen.blit(image, (self.x-20, self.y-20))  # Adjust position to center the image
            except pygame.error as e:
                print(f"Error loading image {self.image}: {e}")


    def calculate_hexagon_points(self, x, y, size):
        points = []
        for i in range(6):
            angle = math.radians(60 * i)
            point_x = x + size * math.cos(angle)
            point_y = y + size * math.sin(angle)
            points.append((point_x, point_y))
        return points


def create_hex_grid_pygame(nx, ny):
    hex_centers, _ = create_hex_grid(nx=nx, ny=ny, do_plot=False)
    grid = []
    
    # Calculate offsets to center the grid
    min_x = min(center[0] for center in hex_centers)
    max_x = max(center[0] for center in hex_centers)
    min_y = min(center[1] for center in hex_centers)
    max_y = max(center[1] for center in hex_centers)
    
    grid_width = ((max_x - min_x) * 1)
    grid_height =((max_y - min_y) * 1)
    
    offset_x = (WIDTH - grid_width) / 2
    offset_y = (HEIGHT - grid_height) / 2
    
    for center in hex_centers:
        x, y = center
        grid.append(Hexagon(x * 40 + offset_x, y * 40 + offset_y))  # Adjust scaling and position as needed
    return grid


def assign_neighbors(grid):
    for hexagon in grid:
        for neighbor in grid:
            if hexagon != neighbor and math.dist((hexagon.x, hexagon.y), (neighbor.x, neighbor.y)) < 50:
                hexagon.neighbors.append(neighbor)


def initialize_pokemon(grid):
    pokemon_types = list(POKEMON_IMAGES.keys())
    random.shuffle(pokemon_types)
    for pokemon_type in pokemon_types:
        hexagon = random.choice(grid)
        while hexagon.type is not None:
            hexagon = random.choice(grid)
        hexagon.type = pokemon_type
        hexagon.image = POKEMON_IMAGES[pokemon_type]
        hexagon.colored_by = pokemon_type

        
def move_pokemon(grid):
    for hexagon in grid:
        if hexagon.type:
            target = random.choice(hexagon.neighbors)
            if target.type:
                if target.type in OVERPOWER[hexagon.type]:
                    # Overpowering logic
                    target.type = hexagon.type
                    target.image = POKEMON_IMAGES[hexagon.type]
                    target.colored_by = hexagon.type
                    hexagon.type = None
                    hexagon.image = None
                # Remove the else block to prevent exchanging positions
            else:
                # Move to empty cell
                target.type = hexagon.type
                target.image = POKEMON_IMAGES[hexagon.type]
                target.colored_by = hexagon.type
                hexagon.type = None
                hexagon.image = None


                
def regenerate_pokemon(grid):
    for hexagon in grid:
        if hexagon.type is None and hexagon.colored_by:
            # Ensure the cell is not colored by another type
            if all(h.colored_by != hexagon.colored_by for h in hexagon.neighbors):
                hexagon.type = hexagon.colored_by
                hexagon.image = POKEMON_IMAGES[hexagon.colored_by]

                  
def draw_grid(grid):
    screen.fill((255, 255, 255))
    for hexagon in grid:
        hexagon.draw()
    pygame.display.update()
    

def count_captured_cells(grid):
    counts = {ptype: 0 for ptype in POKEMON_IMAGES.keys()}
    for hexagon in grid:
        if hexagon.colored_by:
            counts[hexagon.colored_by] += 1
    return counts


def check_remaining_types(grid):
    remaining_types = set(hexagon.colored_by for hexagon in grid if hexagon.colored_by)
    if len(remaining_types) == 2:
        type1, type2 = remaining_types
        if type1 not in OVERPOWER[type2] and type2 not in OVERPOWER[type1]:
            return True
    elif len(remaining_types) <= 1:
        return len(remaining_types) <= 1
    return False


def main():
    grid = create_hex_grid_pygame(GRID_SIZE, GRID_SIZE)
    assign_neighbors(grid)
    initialize_pokemon(grid)
    start_time = pygame.time.get_ticks()
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
        current_time = pygame.time.get_ticks()
        if current_time - start_time > 5000:  # 5 seconds delay
            move_pokemon(grid)
            regenerate_pokemon(grid)  # Add this line to regenerate Pokémon
            if check_remaining_types(grid):
                running = False
        draw_grid(grid)
        clock.tick(1)  # Adjust the speed as needed
    counts = count_captured_cells(grid)
    print("Simulation ended. Captured cells count:")
    for ptype, count in counts.items():
        print(f"{ptype}: {count}")
    pygame.quit()

if __name__ == "__main__":
    main()

In [ ]:
pygame.quit()